In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

# LSTM’s in Pytorch

In [12]:
IN_DIM = 2
OUT_DIM = 4
SEQ_NUM = 6

lstm = nn.LSTM(IN_DIM, OUT_DIM)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, IN_DIM) for _ in range(SEQ_NUM)]  # make a sequence of length 5

# initialize the hidden state.
# (h_t, c_t)
hidden = (torch.randn(1, 1, OUT_DIM),
          torch.randn(1, 1, OUT_DIM))
print('Input shape: ', inputs[0].shape)
print('Input shape: ', inputs[0].view(1, 1, -1).shape)

for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    # .view(1, 1, -1) is to add dimension to match lstm
    out, hidden = lstm(i.view(1, 1, -1), hidden)
print('out: ', out, '\n', out.shape, '\n')
print('hidden', hidden, '\n', hidden[0].shape, '\n', hidden[1].shape, '\n')
# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
print(len(inputs))
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
print('Input shape: ', inputs.shape)
hidden = (torch.randn(1, 1, OUT_DIM), 
          torch.randn(1, 1, OUT_DIM))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print('out: ', out, '\n', out.shape, '\n')
print('hidden', hidden, '\n', hidden[0].shape, '\n', hidden[1].shape)

Input shape:  torch.Size([1, 2])
Input shape:  torch.Size([1, 1, 2])
out:  tensor([[[-0.2216,  0.1439,  0.1821,  0.4131]]], grad_fn=<StackBackward>) 
 torch.Size([1, 1, 4]) 

hidden (tensor([[[-0.2216,  0.1439,  0.1821,  0.4131]]], grad_fn=<StackBackward>), tensor([[[-1.0823,  0.2689,  0.4823,  1.2465]]], grad_fn=<StackBackward>)) 
 torch.Size([1, 1, 4]) 
 torch.Size([1, 1, 4]) 

6
Input shape:  torch.Size([6, 1, 2])
out:  tensor([[[-0.1930, -0.2514, -0.0958,  0.0184]],

        [[-0.2429, -0.1241, -0.1359,  0.0652]],

        [[-0.1565,  0.0280,  0.0146,  0.2853]],

        [[-0.2224,  0.1704,  0.0819,  0.3496]],

        [[-0.1347, -0.0153,  0.3077,  0.5081]],

        [[-0.2196,  0.1235,  0.1779,  0.4062]]], grad_fn=<StackBackward>) 
 torch.Size([6, 1, 4]) 

hidden (tensor([[[-0.2196,  0.1235,  0.1779,  0.4062]]], grad_fn=<StackBackward>), tensor([[[-1.0873,  0.2294,  0.4689,  1.1933]]], grad_fn=<StackBackward>)) 
 torch.Size([1, 1, 4]) 
 torch.Size([1, 1, 4])


# Example: An LSTM for Part-of-Speech Tagging

### Prepare data:

In [87]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

{'that': 7, 'book': 8, 'dog': 1, 'Everybody': 5, 'read': 6, 'the': 3, 'ate': 2, 'The': 0, 'apple': 4}


### Create the model:

In [81]:
# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

### Train the model:

In [104]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-1.4147, -0.8137, -1.1591],
        [-1.4611, -0.8696, -1.0530],
        [-1.4784, -0.9868, -0.9182],
        [-1.4104, -0.9306, -1.0171],
        [-1.3232, -0.8865, -1.1344]])
tensor([[-0.2771, -1.6509, -2.9930],
        [-4.7449, -0.0198, -4.5173],
        [-2.9024, -3.6690, -0.0838],
        [-0.0411, -4.7749, -3.4483],
        [-4.7980, -0.0253, -4.0897]])
